In [1]:
import progNotebook
import datasource
import BinnedScat
import json
import numpy as np
from fast_histogram import histogram1d
import multiprocessing
import matplotlib.pyplot as plt

In [4]:
##QUANTILE EXAMPLES
def updateFunction(scatData,marginalData,progressData):        
    widget.scatData = json.dumps(scatData)
    widget.marginals = json.dumps(marginalData)
    widget.progress = json.dumps(progressData)

#
# scat = BinnedScat.MeanBinScat(updateFunction,'B', 'C', 'A', 3, 3, 6)
# source = datasource.DataSource("numbers2.txt",scat.processChunk,2000)  
    
    
# #
# scat = BinnedScat.QuantileBinScat(updateFunction,'B', 'C', 'A', 10, 10, 20)
# source = datasource.DataSource("data/sample.csv",scat.processChunk,10)  

# #
# scat = BinnedScat.QuantileBinScat(updateFunction,'B', 'C', 'A', 10, 10, 20)
# source = datasource.DataSource("data/bigfile_multiscale.csv",scat.processChunk,10000, 0.01)

# #
# scat = BinnedScat.QuantileBinScat(updateFunction,'DEP_DELAY', 'ARR_DELAY', 'DISTANCE', 30, 30, 20)
# source = datasource.DataSource("data/flightsDataset.csv",scat.processChunk,10000,.1)  

#
scat = BinnedScat.QuantileBinScat(updateFunction,'trip_distance', 'fare_amount', 'tip_amount', 30, 30, 20)
source = datasource.DataSource("data/sampleTaxi.csv",scat.processChunk,1000,.3)  


widget = progNotebook.ProgScatterWidget(source)
display(widget)
widget.start()

ProgScatterWidget()

In [ ]:
##MEAN EXAMPLES
def updateFunction(scatData,marginalData,progressData):        
    widget.scatData = json.dumps(scatData)
    widget.marginals = json.dumps(marginalData)
    widget.progress = json.dumps(progressData)

#
# scat = BinnedScat.MeanBinScat(updateFunction,'B', 'C', 'A', 3, 3, 6)
# source = datasource.DataSource("numbers2.txt",scat.processChunk,2000)  
    
    
#
scat = BinnedScat.MeanBinScat(updateFunction,'B', 'C', 'A', 10, 10, 20)
source = datasource.DataSource("data/sample.csv",scat.processChunk,10)  

# #
# scat = BinnedScat.MeanBinScat(updateFunction,'B', 'C', 'A', 10, 10, 20)
# source = datasource.DataSource("data/bigfile_multiscale.csv",scat.processChunk,10000, 0.01)

#
scat = BinnedScat.MeanBinScat(updateFunction,'DEP_DELAY', 'ARR_DELAY', 'DISTANCE', 30, 30, 20)
source = datasource.DataSource("data/flightsDataset.csv",scat.processChunk,10000,.1)  

# #
# scat = BinnedScat.MeanBinScat(updateFunction,'trip_distance', 'fare_amount', 'tip_amount', 30, 30, 20)
# source = datasource.DataSource("data/sampleTaxi.csv",scat.processChunk,1000,.3)  


widget = progNotebook.ProgScatterWidget(source)
display(widget)
widget.start()

In [ ]:
##COUNT EXAMPLES
def updateFunction(scatData,marginalData,progressData):        
    widget.scatData = json.dumps(scatData)
    widget.marginals = json.dumps(marginalData)
    widget.progress = json.dumps(progressData)

#
# scat = BinnedScat.CountBinScat(updateFunction,'B', 'C', 'A', 3, 3, 6)
# source = datasource.DataSource("numbers2.txt",scat.processCountChunk,2000)  
    
    
# #
# scat = BinnedScat.CountBinScat(updateFunction,'B', 'C', 'A', 10, 10, 20)
# source = datasource.DataSource("data/sample.csv",scat.processCountChunk,2000)  

# #
# scat = BinnedScat.CountBinScat(updateFunction,'B', 'C', 'A', 10, 10, 20)
# source = datasource.DataSource("data/bigfile_multiscale.csv",scat.processCountChunk,10000, 0.01)

#
scat = BinnedScat.CountBinScat(updateFunction,'DEP_DELAY', 'ARR_DELAY', 'DISTANCE', 50, 50, 20)
source = datasource.DataSource("data/flightsDataset.csv",scat.processCountChunk,10000)  

# #
# scat = BinnedScat.CountBinScat(updateFunction,'trip_distance', 'fare_amount', 'trip_distance', 30, 30, 20)
# source = datasource.DataSource("data/sampleTaxi.csv",scat.processCountChunk,1000,.3)  


widget = progNotebook.ProgScatterWidget(source)
display(widget)
widget.start()

In [ ]:
help(datasource.DataSource)

In [ ]:
widget = progNotebook.ProgScatterWidget(source,scat)
display(widget)
widget.start()

In [ ]:
def processCountChunk(chunk,progress):
    scat.updateWithDataFrame(chunk, 'B', 'C', 'A')
    valueBB = scat.getAggregationRange()
    summary = scat.getSummary()
    # build data to pass to widget
    if summary:
        bbox = scat.getFixedBoundingBox()
        scatData = {'xDomainRange': bbox[0],
                    'yDomainRange': bbox[1],
                    'zDomainRange': valueBB,
                    'xRes': scat._resX,
                    'yRes': scat._resY,
                    'xLabel': 'xAxis',
                    'yLabel': 'yAxis',
                    'zLabel': 'zLabel',
                    'sparse': 0,
                    'bins': summary[0].tolist(),
                    'counts': summary[1].tolist()
                    }

        #
        histResolution = 10
        pointsX = scat.getMarginalHistogram(BinnedScat.Axis.X,histResolution)
        pointsX = [[pointsX[0][i],pointsX[1][i]] for i in range(len(pointsX[0]))]
        #
        pointsY = scat.getMarginalHistogram(BinnedScat.Axis.Y,histResolution)
        pointsY = [[pointsY[0][i],pointsY[1][i]] for i in range(len(pointsY[0]))]
        #
        zHist = scat.getCountHistogram()
        numSplits = 10 + 1
        xMin = zHist['range'][0]
        xMax = zHist['range'][1]
        step = (xMax - xMin) / numSplits
        splits = [xMin + (i*step) for i in range(0, numSplits)]
        pointsZ = list(zip(splits,list(zHist['values'])))
        #
        mx = {'points': pointsX, 'xLabel': 'Marginal X'}        
        my = {'points': pointsY, 'xLabel': 'Marginal Y'}
        mz = {'points': pointsZ, 'xLabel': 'Marginal Z'}
        obj = {"mx":mx,"my":my,"mz":mz}
        myProgress = {'current': progress[0], 'max': progress[1]}
        #
        try:
            widget.marginals = json.dumps(obj)
            widget.scatData = json.dumps(scatData)
            widget.progress = json.dumps(myProgress)
        except:
            print(scat.getSummary())
            halt
        #print('*******',progress)

source = datasource.DataSource("data/sample.csv",processCountChunk,10)  
scat = BinnedScat.CountBinScat('B', 'C', 'A', 10, 10, 20)
#source.start()

        
# _x = multiprocessing.Process(target=source.start)
# _x.start()
#datasource.DataSource("data/bigfile_multiscale.csv",processCountChunk,10000).start()

In [ ]:
widget.signalRestart()

In [ ]:
def on_value_change(change):
    print('UWOWOWUWOW',change['new'])
widget.observe(on_value_change, names='restart')

In [ ]:
widget.restart = 13

In [ ]:
widget.restart = 0

In [ ]:
def processMeanChunk(chunk):
    scat.updateWithDataFrame(chunk, 'B', 'C', 'A')
    valueBB = scat.getAggregationRange()
    summary = scat.getSummary()
    # build data to pass to widget
    if summary:
        bbox = scat.getFixedBoundingBox()
        scatData = {'xDomainRange': bbox[0],
                    'yDomainRange': bbox[1],
                    'zDomainRange': valueBB,
                    'xRes': scat._resX,
                    'yRes': scat._resY,
                    'xLabel': 'xAxis',
                    'yLabel': 'yAxis',
                    'zLabel': 'zLabel',
                    'sparse': 0,
                    'bins': summary[0],
                    'counts': summary[1]
                    }

        #
        histResolution = 10
        pointsX = scat.getMarginalHistogram(BinnedScat.Axis.X,histResolution)
        pointsX = [[pointsX[0][i],pointsX[1][i]] for i in range(len(pointsX[0]))]
        #
        pointsY = scat.getMarginalHistogram(BinnedScat.Axis.Y,histResolution)
        pointsY = [[pointsY[0][i],pointsY[1][i]] for i in range(len(pointsY[0]))]
        #
        pointsZ = scat.getMarginalHistogram(BinnedScat.Axis.Z,histResolution)
        pointsZ = [[pointsY[0][i],pointsY[1][i]] for i in range(len(pointsY[0]))]
        #
        mx = {'points': pointsX, 'xLabel': 'Marginal X'}
        my = {'points': pointsY, 'xLabel': 'Marginal Y'}
        mz = {'points': pointsZ, 'xLabel': 'Marginal Z'}
        obj = {"mx":mx,"my":my,"mz":mz}
        try:
            widget.marginals = json.dumps(obj)
            #json.dumps(scatData)
            widget.scatData = json.dumps(scatData)
        except:
            print(summary)
            halt
       
scat = BinnedScat.MeanBinScat('B', 'C', 'A', 10, 10, 20)
datasource.DataSource("data/sample.csv",processMeanChunk,10).start()
#datasource.DataSource("data/bigfile_multiscale.csv",processChunk,1000).start()

In [ ]:
# Long running cell
import multiprocessing, time

def network_call(myStop):
    for i in range(20):
        if myStop == True:
            break
        print(i)
        time.sleep(1)

manager = multiprocessing.Manager()
myStop = manager.Value('myStop', True)
        
x = multiprocessing.Process(target=network_call, args=(myStop,))
x.start()

In [ ]:
print(myStop)
myStop = False
print(myStop)

In [ ]:
x.kill()

In [ ]:
f = open('numbers.txt')
array = []
for line in f:
    array.append(float(line))
plt.hist(array)

In [ ]:
f = open('numbers2.txt')
array = []
for line in f:
    array.append(float(line))
plt.hist(array)

In [ ]:
np.array([[1,2], [3,4]]).flatten()